In [20]:
from autogen.agents.experimental import DocAgent
import os
from autogen.agents.experimental import DocAgent
from autogen import ConversableAgent
llm_config = {'cache_seed': 42,
                    'temperature': 0.9,
                    'top_p': 0.05,
                    'config_list': [{'model': 'gpt-4o-mini',
                                    'api_key': os.getenv('OPENAI_API_KEY'),
                                    'api_type': 'openai'}],
                    'timeout': 1200}

DOC_PATH = "data/"
NUMBER = 20
QUERY = f"Create a list of the top {NUMBER} most important concepts. For each concept, explain it in a VERY detailed manner and why it is important."

doc_agent = DocAgent(llm_config=llm_config, collection_name='summarise')
run_response = doc_agent.run(
    message = f"ingest all documents in {DOC_PATH} and do {QUERY}.",
    max_turns=1,
    
)
run_response.process()

result = run_response.messages[1]['content']

finalizer = ConversableAgent(
    name="Finalizer",
    llm_config=llm_config,
    system_message="Your job is to clean up the results from the DocAgent. Return the results in a list format, including as much detail as possible. Return ONLY the list with no other additional text.",
)

finalizer_response = finalizer.run(
    message=result,
    max_turns=1,
)
finalizer_response.process()
final_result = finalizer_response.messages[1]['content']

INFO:autogen.agents.experimental.document_agent.chroma_query_engine:Using existing collection summarise from the database.


user (to DocAgent):

ingest all documents in data/ and do Create a list of the top 20 most important concepts. For each concept, explain it in a VERY detailed manner and why it is important..

--------------------------------------------------------------------------------
_User (to chat_manager):

ingest all documents in data/ and do Create a list of the top 20 most important concepts. For each concept, explain it in a VERY detailed manner and why it is important..

--------------------------------------------------------------------------------

Next speaker: DocumentTriageAgent

DocumentTriageAgent (to chat_manager):

{"ingestions":[{"path_or_url":"data/"}],"queries":[{"query_type":"RAG_QUERY","query":"Create a list of the top 20 most important concepts and explain each in detail."}]}

--------------------------------------------------------------------------------

Next speaker: TaskManagerAgent


>>>>>>>> USING AUTO REPLY...
TaskManagerAgent (to chat_manager):

***** Suggested too

INFO:autogen.agents.experimental.document_agent.document_utils:Detected directory. Listing files...


***** Suggested tool call (call_jsViNrcFA2qJUgQdDI4FLNju): data_ingest_task *****
Arguments: 
{}
*********************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION data_ingest_task...
Call ID: call_jsViNrcFA2qJUgQdDI4FLNju
Input arguments: {}


INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash abf069e0fb0f219e247d3d2b243ad857
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document UnderstandingDeepLearning_DiffusionModels.pdf


KeyboardInterrupt: 